<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/Acme.jpeg width=100 align="left">

# AWS Dynamic Credentials
---

## Business Value of Capability
The AWS secrets engine generates AWS access credentials dynamically based on IAM policies. This generally makes working with AWS IAM easier, since it does not involve clicking in the web UI. Additionally, the process is codified and mapped to internal auth methods (such as LDAP). The AWS IAM credentials are time-based and are automatically revoked when the Vault lease expires.
### Setup
---

In [2]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_TOKEN=root
export VAULT_SKIP_VERIFY=true

In [ ]:
vault login root http://127.0.0.1:8200

In [3]:
vault status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.6.0
Storage Type    inmem_transactional_ha
Cluster Name    vault-cluster-0ccb697d
Cluster ID      e60fbe4d-fcbf-6f14-5f60-a5f54f6e5ca7
HA Enabled      true
HA Cluster      https://127.0.0.1:8201
HA Mode         active


In [ ]:
vault -version

### Demo
---
**Enable the Vault AWS secrets engine**

In [ ]:
vault secrets enable aws

**Write some secrets to Vault**

In [ ]:
vault write aws/config/root \
      access_key=$AWS_ACCESS_KEY \
      secret_key=$AWS_SECRET_ACCESS_KEY

**Write a role to Vault**

In [ ]:
vault write aws/roles/my-role \
        credential_type=iam_user \
        policy_document=-<<EOF
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "Stmt1426528957000",
      "Effect": "Allow",
      "Action": [
        "ec2:*"
      ],
      "Resource": [
        "*"
      ]
    }
  ]
}
EOF

**Reading the role creates the credentials in AWS as an IAM user with those permissions above**

In [ ]:
vault read aws/creds/my-role

In [ ]:
export AWS_ACCESS_KEY_ID=foo
export AWS_SECRET_ACCESS_KEY=bar
aws ec2 describe-regions --filters "Name=endpoint,Values=*us*"

**Revoking the lease removes the user**

In [ ]:
vault lease revoke aws/creds/my-role/<foo>

**Revoke all leases**

In [ ]:
vault lease revoke -prefix=true aws/creds/my-role

**END**